# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [64]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import numpy as np
import cartopy.crs
import cartopy.feature
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [65]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,wailua homesteads,22.0669,-159.3780,25.09,69,75,4.63,US,1683859428
1,1,baringo,0.4667,35.9667,19.11,71,35,2.67,KE,1683859429
2,2,ola,59.5833,151.2833,7.86,76,53,2.71,RU,1683859429
3,3,port elizabeth,-33.9180,25.5701,14.89,91,20,7.72,ZA,1683859429
4,4,papatowai,-46.5619,169.4708,11.30,90,100,4.24,NZ,1683859429


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [66]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE
map_plot_1 = city_data_df.hvplot.points(
        "Lng",
        "Lat",
         geo= True,
         tiles = True,
         frame_width = 800,
         frame_height = 600,
         size = "Humidity",
         scale = 0.9,
         color = "City"
)
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [67]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

# Drop any rows with null values
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE
vacation_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) 
                                  & (city_data_df['Max Temp'] > 21) 
                                  & (city_data_df['Wind Speed'] < 4.5)
                                  & (city_data_df['Cloudiness'] == 0)
                                 ]

# Drop any rows will null values.
vacation_df_complete = vacation_df.dropna()
vacation_df_complete

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,20,port macquarie,-31.4333,152.9167,22.16,61,0,3.53,AU,1683859319
103,103,newman,37.3138,-121.0208,23.18,56,0,2.47,US,1683859473
164,164,remire-montjoly,4.9167,-52.2667,24.02,94,0,1.03,GF,1683859493
330,330,santana,-0.0583,-51.1817,25.10,100,0,1.03,BR,1683859553
401,401,gebeit,21.0667,36.3167,26.18,14,0,1.59,SD,1683859579
402,402,pendleton,45.6721,-118.7886,21.76,43,0,3.60,US,1683859579
424,424,hotan,37.0997,79.9269,22.61,6,0,1.43,CN,1683859587
426,426,brandon,27.9378,-82.2859,26.85,69,0,3.13,US,1683859385
435,435,saint-pierre,-21.3393,55.4781,22.78,60,0,3.60,RE,1683859592
452,452,ajman city,25.4111,55.4350,26.19,31,0,1.03,AE,1683859599


### Step 3: Create a new DataFrame called `hotel_df`.

In [68]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_df_complete[["City", "Country","Lat","Lng","Humidity"] ].copy() # YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
20,port macquarie,AU,-31.4333,152.9167,61,
103,newman,US,37.3138,-121.0208,56,
164,remire-montjoly,GF,4.9167,-52.2667,94,
330,santana,BR,-0.0583,-51.1817,100,
401,gebeit,SD,21.0667,36.3167,14,
402,pendleton,US,45.6721,-118.7886,43,
424,hotan,CN,37.0997,79.9269,6,
426,brandon,US,27.9378,-82.2859,69,
435,saint-pierre,RE,-21.3393,55.4781,60,
452,ajman city,AE,25.4111,55.4350,31,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [69]:
# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude =row["Lat"]# YOUR CODE HERE
    longitude =row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}" # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    print


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json() # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port macquarie - nearest hotel: The Observatory
newman - nearest hotel: No hotel found
remire-montjoly - nearest hotel: Complexe Belova
santana - nearest hotel: CETA Ecotel
gebeit - nearest hotel: No hotel found
pendleton - nearest hotel: Working Girls Hotel
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
brandon - nearest hotel: Brandon Motor Lodge
saint-pierre - nearest hotel: Tropic Hotel
ajman city - nearest hotel: Safari Hotel Apartments
arar - nearest hotel: دار أوتيل Dar Otel
balqash - nearest hotel: Respect


,City,Country,Lat,Lng,Humidity,Hotel Name
20,port macquarie,AU,-31.4333,152.9167,61,The Observatory
103,newman,US,37.3138,-121.0208,56,No hotel found
164,remire-montjoly,GF,4.9167,-52.2667,94,Complexe Belova
330,santana,BR,-0.0583,-51.1817,100,CETA Ecotel
401,gebeit,SD,21.0667,36.3167,14,No hotel found
402,pendleton,US,45.6721,-118.7886,43,Working Girls Hotel
424,hotan,CN,37.0997,79.9269,6,Hua Yi International Hotel (accepts foreigners)
426,brandon,US,27.9378,-82.2859,69,Brandon Motor Lodge
435,saint-pierre,RE,-21.3393,55.4781,60,Tropic Hotel
452,ajman city,AE,25.4111,55.4350,31,Safari Hotel Apartments


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [70]:
#%%capture --no-display

# Configure the map plot
map_plot_2= hotel_df.hvplot.points(
         "Lng",
         "Lat",
         geo= True,
         tiles = True,
         frame_width =800,
         frame_height = 800,
         scale = 0.9,
         hover_cols=['Humidity','Hotel Name','Country'],
         color = "City")

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)